## 직원 현황 : 여러 회사(상장기업)
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/

### 1. DART에서 직원현황 데이터 가져오기
+ report code:  사업보고서(11011), 1분기보고서(11013), 반기보고서(11012), 3분기보고서(11014)
+ 가져오는 데이터: 
   + fo_bbm    사업부문
   + rgllbr_co 정규직 수
   + cnttk_co  계약직 수
   + sm                 합계
   + avrg_cnwk_sdytrn  평균 근속 연수
   + fyer_salary_totamt  연간 급여 총액
   + jan_salary_am     1인평균 급여액

In [21]:
import requests
import pandas as pd

url = 'https://opendart.fss.or.kr/api/empSttus.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'

listed_corp_codes = \
['01208885',
 '00787376',
 '00128555',
 '00130684']

df_all = pd.DataFrame()

for corp_code in listed_corp_codes:
    params = {
        'corp_code': corp_code,
        'bsns_year': 2023,
        'reprt_code': 11012,
        'crtfc_key': api_key,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:     # URL GET '200 정상'
        df_emp = pd.json_normalize(response.json()['list'])
        df_all = pd.concat([df_all, df_emp])
    else:
        print("URL GET Error", corp_code)
        
df_all

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20230814002096,Y,01208885,경동도시가스,남,도시가스,-,-,-,243,-,28,-,271,17년 11개월,"11,476,219,000","42,348,000",-
1,20230814002096,Y,01208885,경동도시가스,여,도시가스,-,-,-,19,-,6,-,25,15년 4개월,"884,912,000","35,396,000",-
0,20230811002127,Y,00787376,대성에너지,남,도시가스,-,-,-,366,-,7,-,373,19년 9개월,"16,812,000,000","45,000,000",-
1,20230811002127,Y,00787376,대성에너지,여,도시가스,-,-,-,30,-,-,-,30,14년 7개월,"987,000,000","33,000,000",-
0,20230811002139,Y,00128555,삼천리,남,도시가스,-,-,-,560,-,8,-,568,17년 11개월,"26,699,000,000","47,000,000",-
1,20230811002139,Y,00128555,삼천리,여,도시가스,-,-,-,43,-,0,-,43,22년 11개월,"1,641,000,000","38,000,000",-
2,20230811002139,Y,00128555,삼천리,남,기타,-,-,-,189,-,3,-,192,11년 10개월,"11,595,000,000","60,000,000",-
3,20230811002139,Y,00128555,삼천리,여,기타,-,-,-,43,-,1,-,44,9년 1개월,"1,864,000,000","42,000,000",-
0,20230814001424,Y,00130684,서울도시가스,남,도시가스공급,-,-,-,470,0,5,0,475,22.5,"27,329,000,000","57,000,000",-
1,20230814001424,Y,00130684,서울도시가스,여,도시가스공급,-,-,-,36,0,6,0,42,16.2,"1,491,000,000","35,000,000",-


### 2. 필요한 데이터만 추출

In [22]:
df_all[['corp_code', 'corp_name', 'fo_bbm', 'sexdstn', 'sm', 'rgllbr_co', 'cnttk_co', 'avrg_cnwk_sdytrn']]
df_all['cnttk_co'].replace({'-': '0'}, inplace=True)
df_all['rgllbr_co'].replace({'-': '0'}, inplace=True)
df_all['sm'].replace({'-': '0'}, inplace=True)
corp_employee = df_all.astype({'sm' : 'int',                               
                               'rgllbr_co' : 'int',
                               'cnttk_co' : 'int'})
df_emp = corp_employee[['corp_code', 'corp_name', 'fo_bbm', 'sexdstn', 'sm', 'rgllbr_co', 'cnttk_co', 'avrg_cnwk_sdytrn']]
df_emp

,corp_code,corp_name,fo_bbm,sexdstn,sm,rgllbr_co,cnttk_co,avrg_cnwk_sdytrn
0,01208885,경동도시가스,도시가스,남,271,243,28,17년 11개월
1,01208885,경동도시가스,도시가스,여,25,19,6,15년 4개월
0,00787376,대성에너지,도시가스,남,373,366,7,19년 9개월
1,00787376,대성에너지,도시가스,여,30,30,0,14년 7개월
0,00128555,삼천리,도시가스,남,568,560,8,17년 11개월
1,00128555,삼천리,도시가스,여,43,43,0,22년 11개월
2,00128555,삼천리,기타,남,192,189,3,11년 10개월
3,00128555,삼천리,기타,여,44,43,1,9년 1개월
0,00130684,서울도시가스,도시가스공급,남,475,470,5,22.5
1,00130684,서울도시가스,도시가스공급,여,42,36,6,16.2


### 3. 엑셀로 저장

In [23]:
df_emp.to_excel('D:\PythonProject\data-gatherer\dart\data\직원현황.xlsx')

### 4. 그룹핑

In [24]:
corp_employee.groupby(['corp_code', 'corp_name']).agg(sm = ('sm', 'sum'),
                                                      rgllbr_co = ('rgllbr_co', 'sum'),
                                                      cnttk_co = ('cnttk_co', 'sum'))

,,sm,rgllbr_co,cnttk_co
corp_code,corp_name,,,
00128555,삼천리,847,835,12
00130684,서울도시가스,517,506,11
00787376,대성에너지,403,396,7
01208885,경동도시가스,296,262,34
